## Tutorial showing how to use DCM

### Setup

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pytorch_lightning as pl

import torch
from model.model_dcm import ModelDCM
from modules.cell_network import CellNetwork
from torch_geometric import seed_everything
from torch_geometric.data import LightningNodeData
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

torch.set_default_tensor_type("torch.cuda.FloatTensor")
torch.set_float32_matmul_precision("high")


This configuration file is used to determine the hyperparameters of the model and set the random seeds to have repeatable results.

In [3]:
config = {
    "metric": {"name": "val_acc", "goal": "maximize"},
    "seed":  21,
    "data_seed":  0,
    "hsize":  32,
    "n_pre":  1,
    "n_post":  1,
    "n_conv":  1,
    "n_dgm_layers":  2,
    "dropout":  0.5,
    "lr":  0.01,
    "use_gcn":  False,
    "k":  4,
    "graph_loss_reg":  1,
    "poly_loss_reg":  1,
    "std":  0,
    "ensemble_steps":  3,
    "gamma":  50,
    "epochs":  200,
    }

seed_everything(config["seed"])

### Loading the dataset

For this example the Cora dataset is used.

In [4]:
dataset = Planetoid(
        root="data/Planetoid",
        name="cora",
        split="random",
        num_train_per_class=373,
        num_val=271,
        num_test=271,
        transform=NormalizeFeatures(),
    )
    
data = dataset[0]

datamodule = LightningNodeData(
    data,
    data.train_mask,
    data.val_mask,
    data.test_mask,
    loader="full",
    generator=torch.Generator(device="cuda"),
)

### Creating the model

In [5]:
config["num_features"] = dataset.num_features
config["num_classes"] = dataset.num_classes
config["pre_layers"] = [dataset.num_features] + [config["hsize"] for _ in range(config["n_pre"])]
config["post_layers"] = [config["hsize"] for _ in range(config["n_post"])] + [dataset.num_classes]
config["dgm_layers"] = [config["hsize"] for _ in range(config["n_dgm_layers"] + 1)]
config["conv_layers"] = [config["hsize"] for _ in range(config["n_conv"])]

model = ModelDCM(config)

### Training

In [6]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_acc", mode="max")

trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    max_epochs=config["epochs"],
    log_every_n_steps=1,
    check_val_every_n_epoch=3,
    num_sanity_val_steps=1,
    callbacks=[checkpoint_callback],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model, datamodule)
trainer.test(ckpt_path="best", datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | pre          | MLP                | 45.9 K
1 | dcm          | DCM                | 2.1 K 
2 | cell_network | CellNetwork        | 0     
3 | post         | MLP                | 455   
4 | train_acc    | MulticlassAccuracy | 0     
5 | val_acc      | MulticlassAccuracy | 0     
6 | test_acc     | MulticlassAccuracy | 0     
----------------------------------------------------
48.5 K    Trainable params
0         Non-trainable params
48.5 K    Total params
0.194     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  5.41it/s, loss=0.377, v_num=49, train_acc=0.873, train_loss=0.361, val_acc=0.727]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  5.08it/s, loss=0.377, v_num=49, train_acc=0.873, train_loss=0.361, val_acc=0.727]


Restoring states from the checkpoint path at /home/marco/Documents/phd/DCM_simple/differentiable_cell-complex_module/lightning_logs/version_49/checkpoints/epoch=50-step=51.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/marco/Documents/phd/DCM_simple/differentiable_cell-complex_module/lightning_logs/version_49/checkpoints/epoch=50-step=51.ckpt


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.73it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7195571660995483
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.7195571660995483}]

### Changing the CellNetwork

Testing a different network over the structure inferred by DCM is straightforward.

In [8]:
# addign one more conv layer
config["num_features"] = dataset.num_features
config["num_classes"] = dataset.num_classes
config["pre_layers"] = [dataset.num_features] + [config["hsize"] for _ in range(config["n_pre"])]
config["post_layers"] = [config["hsize"] for _ in range(config["n_post"])] + [dataset.num_classes]
config["dgm_layers"] = [config["hsize"] for _ in range(config["n_dgm_layers"] + 1)]
config["conv_layers"] = [config["hsize"] for _ in range(config["n_conv"] + 1)]

model = ModelDCM(config)

new_cell_network = CellNetwork(config)
model.cell_network = new_cell_network

In [9]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_acc", mode="max")

trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    max_epochs=config["epochs"],
    log_every_n_steps=1,
    check_val_every_n_epoch=3,
    num_sanity_val_steps=1,
    callbacks=[checkpoint_callback],
)
trainer.fit(model, datamodule)
trainer.test(ckpt_path="best", datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | pre          | MLP                | 45.9 K
1 | dcm          | DCM                | 2.1 K 
2 | cell_network | CellNetwork        | 4.2 K 
3 | post         | MLP                | 455   
4 | train_acc    | MulticlassAccuracy | 0     
5 | val_acc      | MulticlassAccuracy | 0     
6 | test_acc     | MulticlassAccuracy | 0     
----------------------------------------------------
52.7 K    Trainable params
0         Non-trainable params
52.7 K    Total params
0.211     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  7.49it/s, loss=0.697, v_num=50, train_acc=0.779, train_loss=0.676, val_acc=0.742]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  6.95it/s, loss=0.697, v_num=50, train_acc=0.779, train_loss=0.676, val_acc=0.742]

Restoring states from the checkpoint path at /home/marco/Documents/phd/DCM_simple/differentiable_cell-complex_module/lightning_logs/version_50/checkpoints/epoch=5-step=6.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/marco/Documents/phd/DCM_simple/differentiable_cell-complex_module/lightning_logs/version_50/checkpoints/epoch=5-step=6.ckpt


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.87it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8339483141899109
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.8339483141899109}]